In [95]:
from datetime import datetime, timedelta
import pandas as pd
from statistics import mean

## Code below formats pax itineratary data for May 2nd

In [ ]:
# Read in the pax itineratary data
pax_itinerary = pd.read_csv('Data/Large_Pax_Itineraries.csv')

# Remove the leading space in each of the column names
pax_itinerary.columns = pax_itinerary.columns.str.strip()

# Filter out the rows where 'month' is not 5 and 'day' is not 2
pax_itinerary = pax_itinerary[(pax_itinerary['month'] == 5) & (pax_itinerary['day_of_month'] == 2)]

# print first 5 rows
print(pax_itinerary.head())

# Save the dataframe as a CSV file
pax_itinerary.to_csv('Data/Pax_Itineraries_May_2nd.csv', index=False)

In [8]:
def time_to_minutes(time_str):
    if time_str is None:
        return None
    
    if time_str == "Inf":
        return float("inf")
    
    if isinstance(time_str, pd.Timestamp):
        time_str = time_str.strftime("%H%M")
    
    # Ensure the time_str is 4 characters long (e.g., "720" becomes "0720")
    time_str = time_str.zfill(4)
    
    hours = int(time_str[:2]) 
    minutes = int(time_str[2:])
    return hours * 60 + minutes


In [9]:
def get_formatted_time(time_str):
    if time_str is None or pd.isna(time_str):
        return None
    else:
        return time_str.strftime("%H%M")

## Cleaning flight and itinerary data to input into optimization model

In [ ]:
import pandas as pd

# Load the data
file_path_flights = "Data/Sample_Day_May_2nd.txt"
file_path_pax = "Data/Pax_Itineraries_May_2nd.csv"

df_flights = pd.read_csv(file_path_flights, delimiter='\t')
df_pax = pd.read_csv(file_path_pax)

# Filter out the rows where SchedDepApt or SchedArrApt is not DFW
df_flights = df_flights[(df_flights['SchedDepApt'] == 'DFW') | (df_flights['SchedArrApt'] == 'DFW')]

# Filter out the rows where DepGate has no value
df_flights = df_flights[df_flights['DepGate'].notna()]

# Remove the rows where DepGate does not start with 'A', 'B', or 'C'
df_flights = df_flights[df_flights['DepGate'].str.startswith(('A', 'B', 'C'))]

# Initialize the connecting passengers column to 0
df_flights['num_connecting_pax'] = 0

# Create a flight index
df_flights['flight_index'] = range(1, len(df_flights) + 1)

# Initialize the connections matrix
connections_matrix = pd.DataFrame(0, index=df_flights['flight_index'], columns=df_flights['flight_index'])

# Convert time columns to datetime
df_flights['SchedDepUtc'] = pd.to_datetime(df_flights['SchedDepUtc'], errors='coerce')
df_flights['SchedArrUtc'] = pd.to_datetime(df_flights['SchedArrUtc'], errors='coerce')
df_pax['planned_connection_time'] = pd.to_datetime(df_pax['planned_connection_time'], errors='coerce')
df_pax['planned_arrival_time'] = pd.to_datetime(df_pax['planned_arrival_time'], errors='coerce')

# Define time tolerance for matching (5 minutes)
time_tolerance = pd.Timedelta(minutes=5)

# Loop through each flight
def populate_connections_matrix():
    for flight_idx, flight_row in df_flights.iterrows():
        if flight_row['SchedDepApt'] == "DFW":
            pax_rows = df_pax[(df_pax['planned_destination'] == flight_row['SchedArrApt']) &
                              (abs(df_pax['planned_connection_time'] - flight_row['SchedDepUtc']) <= time_tolerance)]
            if not pax_rows.empty:
                for _, pax_row in pax_rows.iterrows():
                    inbound_match = df_flights[(df_flights['SchedArrApt'] == "DFW") &
                                               (abs(df_flights['SchedArrUtc'] - pax_row['planned_arrival_time']) <= time_tolerance) &
                                               (df_flights['SchedDepApt'] == pax_row['planned_origin'])]
                    if not inbound_match.empty:
                        inbound_flight_index = inbound_match['flight_index'].iloc[0]
                        connections_matrix.at[inbound_flight_index, flight_row['flight_index']] += pax_row['num_passengers']
                df_flights.at[flight_idx, 'num_connecting_pax'] = pax_rows['num_passengers'].sum()

        elif flight_row['SchedArrApt'] == "DFW":
            pax_rows = df_pax[(df_pax['planned_origin'] == flight_row['SchedDepApt']) &
                              (abs(df_pax['planned_arrival_time'] - flight_row['SchedArrUtc']) <= time_tolerance)]
            if not pax_rows.empty:
                for _, pax_row in pax_rows.iterrows():
                    outbound_match = df_flights[(df_flights['SchedDepApt'] == "DFW") &
                                                (abs(df_flights['SchedDepUtc'] - pax_row['planned_connection_time']) <= time_tolerance) &
                                                (df_flights['SchedArrApt'] == pax_row['planned_destination'])]
                    if not outbound_match.empty:
                        outbound_flight_index = outbound_match['flight_index'].iloc[0]
                        connections_matrix.at[flight_row['flight_index'], outbound_flight_index] += pax_row['num_passengers']
                df_flights.at[flight_idx, 'num_connecting_pax'] = pax_rows['num_passengers'].sum()

# Run the function
populate_connections_matrix()

# Save the updated connections matrix
connections_matrix.to_csv('Data/connections_matrix.csv')
small_connections_matrix = connections_matrix.iloc[0:100, 0:100]
small_connections_matrix.to_csv('Data/small_connections_matrix.csv')


raw_df = df_flights

formatted_path = "Data/Formatted_Sample_Day_May_2nd.csv"
formatted_df = pd.read_csv(formatted_path)

raw_df['CombinedFlightNumber'] = raw_df['AirlineCode'].astype(str) + raw_df['FlightNumber'].astype(str)

merged_df = pd.merge(raw_df, formatted_df[['Flight Number', 'Aircraft', 'Flight Type']], 
                left_on='CombinedFlightNumber', right_on='Flight Number', how='left')

# Now, propagate the 'Aircraft' values across rows with matching 'TailNumber'
merged_df['Aircraft'] = merged_df.groupby('TailNumber')['Aircraft'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))

merged_df['SchedDepLocal'] = pd.to_datetime(merged_df['SchedDepLocal'], format="%Y-%m-%d %H:%M")
merged_df['SchedArrLocal'] = pd.to_datetime(merged_df['SchedArrLocal'], format="%Y-%m-%d %H:%M")
merged_df['InGateLocal'] = pd.to_datetime(merged_df['InGateLocal'], format="%Y-%m-%d %H:%M")
merged_df['OutGateLocal'] = pd.to_datetime(merged_df['OutGateLocal'], format="%Y-%m-%d %H:%M")

new_formatted_data = []
for _, row in merged_df.iterrows():
    is_departing = row['SchedDepApt'] == 'DFW'
    flight_number = row['CombinedFlightNumber']
    destination = row['SchedArrApt']
    origin = row['SchedDepApt']
    aircraft = row['Aircraft']
    flight_type = row['Flight Type']
    if is_departing:
        DepGate = row['DepGate']
        ArrGate = ""
    else:
        DepGate = ""
        ArrGate = row['ArrGate']
    

    # Check if the arrival time of the aircraft was within 2 hours, else assign it to 90 min before departure
    if is_departing:
        off_time = get_formatted_time(row['SchedDepLocal'])
        off_time_actual = get_formatted_time(row['OutGateLocal'])

        arrival_row = merged_df[(merged_df['TailNumber'] == row['TailNumber']) &
                                (merged_df['SchedArrApt'] == 'DFW') &
                                (row['SchedDepLocal'] - merged_df['SchedArrLocal'] <= timedelta(hours=2)) &
                                (merged_df['SchedArrLocal'] < row['SchedDepLocal'])]
        arrival_row = arrival_row.sort_values(by='SchedArrLocal', ascending=False)

        if not arrival_row.empty:
            arrival_time = get_formatted_time(arrival_row.iloc[0]['SchedArrLocal'])
        else:
            arrival_time = get_formatted_time(row['SchedDepLocal'] - timedelta(minutes=90))

        arrival_time_actual_row = merged_df[(merged_df['TailNumber'] == row['TailNumber']) &
                                            (merged_df['SchedArrApt'] == 'DFW') &
                                            (row['SchedDepLocal'] - merged_df['InGateLocal'] <= timedelta(hours=2)) &
                                            (merged_df['InGateLocal'] < row['SchedDepLocal'])]
        arrival_time_actual_row = arrival_time_actual_row.sort_values(by='InGateLocal', ascending=False)

        if not arrival_time_actual_row.empty:
            arrival_time_actual = get_formatted_time(arrival_time_actual_row.iloc[0]['InGateLocal'])
        else:
            arrival_time_actual = get_formatted_time(row['SchedDepLocal'] - timedelta(minutes=90))

        passengers_arr = 0  # No arriving passengers in departing flights
        passengers_dept = {1: 300, 2: 150, 3: 75}.get(aircraft, 0)

    #If arriving flight, check if the aircraft is departing within the next 2 hours, else assign 30 min after arrival
    else:
        arrival_time = get_formatted_time(row['SchedArrLocal'])
        arrival_time_actual = get_formatted_time(row['InGateLocal'])

        off_row = merged_df[(merged_df['TailNumber'] == row['TailNumber']) &
                                (merged_df['SchedDepApt'] == 'DFW') &
                                (merged_df['SchedDepLocal'] - row['SchedArrLocal'] <= timedelta(hours=2)) &
                                (merged_df['SchedDepLocal'] > row['SchedArrLocal'])]
        off_row = off_row.sort_values(by='SchedDepLocal', ascending=True)

        if not off_row.empty:
            off_time = get_formatted_time(off_row.iloc[0]['SchedDepLocal'])
        else:
            off_time = get_formatted_time(row['SchedArrLocal'] + timedelta(minutes=30))

        off_time_actual_row = merged_df[(merged_df['TailNumber'] == row['TailNumber']) &
                                            (merged_df['SchedDepApt'] == 'DFW') &
                                            (merged_df['OutGateLocal'] - row['SchedArrLocal'] <= timedelta(hours=4)) &
                                            (merged_df['OutGateLocal'] > row['SchedArrLocal'])]
        off_time_actual_row = off_time_actual_row.sort_values(by='OutGateLocal', ascending=True)

        if not off_time_actual_row.empty:
            off_time_actual = get_formatted_time(off_time_actual_row.iloc[0]['OutGateLocal'])
        else:
            off_time_actual = get_formatted_time(row['SchedArrLocal'] + timedelta(minutes=30))

        passengers_arr = {1: 300, 2: 150, 3: 75}.get(aircraft, 0)
        passengers_dept = 0         # No departing passengers in arriving flights

    new_formatted_data.append({
        "FlightNumber": flight_number,
        "IsDeparting": 'Y' if is_departing else 'N',
        "Destination": destination,
        "Origin": origin,
        "ArrivalTime": arrival_time,
        "OffTime": off_time,
        "ArrivalTimeMinutes": time_to_minutes(arrival_time),
        "OffTimeMinutes": time_to_minutes(off_time),
        "ArrivalTimeActual": arrival_time_actual,
        "OffTimeActual": off_time_actual,
        "ArrivalTimeActualMinutes": time_to_minutes(arrival_time_actual),
        "OffTimeActualMinutes": time_to_minutes(off_time_actual),
        "Aircraft": aircraft,
        "FlightType": flight_type,
        "PassengersArr": passengers_arr,
        "PassengersDept": passengers_dept,
        "DepGate": DepGate,
        "ArrGate": ArrGate,
        "SchedDepLocal": row['SchedDepLocal'],
        "SchedArrLocal": row['SchedArrLocal'],
        "OutGateLocal": row['OutGateLocal'],
        "InGateLocal": row['InGateLocal'],
        "SchedDepUtc": row['SchedDepUtc'],
        "SchedArrUtc": row['SchedArrUtc'],
        "OutGateUtc": row['OutGateUtc'],
        "InGateUtc": row['InGateUtc'],
        "TailNumber": row['TailNumber'],
        "num_connecting_pax": row['num_connecting_pax']
    })

# if the departure time is less than the arrival time, set the departure time to the arrival time + 24 hours
for row in new_formatted_data:
    if row['OffTimeMinutes'] is not None and row['ArrivalTimeMinutes'] is not None and row['OffTimeMinutes'] < row['ArrivalTimeMinutes']:
        row['OffTimeMinutes'] = row['OffTimeMinutes'] + 24 * 60

# if the actual departure time is less than the actual arrival time, set the actual departure time to the actual arrival time + 24 hours
for row in new_formatted_data:
    if row['OffTimeActualMinutes'] is not None and row['ArrivalTimeActualMinutes'] is not None and row['OffTimeActualMinutes'] < row['ArrivalTimeActualMinutes']:
        row['OffTimeActualMinutes'] = row['OffTimeActualMinutes'] + 24 * 60

new_formatted_df = pd.DataFrame(new_formatted_data)

new_formatted_df = new_formatted_df.sort_values(by='ArrivalTimeMinutes')

new_formatted_df.loc[new_formatted_df['IsDeparting'] == 'Y', 'PassengersDept'] -= new_formatted_df['num_connecting_pax']
new_formatted_df.loc[new_formatted_df['IsDeparting'] != 'Y', 'PassengersArr'] -= new_formatted_df['num_connecting_pax']

# Clip PassengersDept and PassengersArr to 0
new_formatted_df['PassengersDept'] = new_formatted_df['PassengersDept'].clip(lower=0)
new_formatted_df['PassengersArr'] = new_formatted_df['PassengersArr'].clip(lower=0)

new_formatted_df.to_csv("Data/Final_Formatted_Sample_Day.csv", index=False)

# Create df which is first 100 rows of new_formatted_df
new_formatted_df_small = new_formatted_df.head(100)

# save to csv
new_formatted_df_small.to_csv("Data/Small_Final_Formatted_Sample_Day.csv", index=False)


## Walking distance metrics based on actual 2016 gate assignments

In [5]:
import numpy as np
import pandas as pd
print("\nMetrics based on actual 2016 gate assignments:")

# Read in the flight data and walking distances for arriving and departing passengers
file_path = "ConnObj_Optimized_Gate_Assignments_Sample_Day.csv"
df = pd.read_csv(file_path)
walking_distances = pd.read_csv("Data/Walking Distances Arriving and Departing Pax.csv")
print(walking_distances.columns)

# Filter df to include only rows with valid gate assignments based on walking distances
orig_rows = len(df)
valid_gates = set(walking_distances["Gate_Name"])
valid_indices = df.index[df["OptDepGate"].isin(valid_gates) | df["OptArrGate"].isin(valid_gates)].tolist()
df = df.loc[valid_indices]
filtered_out = orig_rows - len(df)
print("Rows filtered out because gate assignment was not in terminals A, B, or C:", filtered_out)
print("Rows remaining in df:", len(df))
print("Indices of filtered out rows:", [i for i in range(orig_rows) if i not in valid_indices])

# Initialize accumulators for departing and arriving walking distances and passenger counts
total_departing_wd = 0.0
total_arriving_wd = 0.0
total_departing_passengers = 0
total_arriving_passengers = 0

# Process each flight row for departing and arriving distances
for _, row in df.iterrows():
    if row["IsDeparting"] == "Y":
        # Use DepGate for departing flights
        gate = row["OptDepGate"]
        gate_match = walking_distances[ walking_distances["Gate_Name"] == gate ]
        if gate_match.empty:
            print(f"WARNING: Departing gate {gate} not found in walking_distances.")
            continue
        distance = gate_match.iloc[0]["TSA_to_Gate"]
        total_departing_wd += row["PassengersDept"] * distance
        total_departing_passengers += row["PassengersDept"]
    else:
        # Use ArrGate for arriving flights
        gate = row["OptArrGate"]
        gate_match = walking_distances[ walking_distances["Gate_Name"] == gate ]
        if gate_match.empty:
            print(f"WARNING: Arriving gate {gate} not found in walking_distances.")
            continue
        distance = gate_match.iloc[0]["Gate_to_Bag"]
        total_arriving_wd += row["PassengersArr"] * distance
        total_arriving_passengers += row["PassengersArr"]

avg_departing_wd = total_departing_wd / total_departing_passengers if total_departing_passengers else 0
avg_arriving_wd = total_arriving_wd / total_arriving_passengers if total_arriving_passengers else 0

# Load the connections matrix and convert it to a NumPy array for easier indexing
conn_mat_df = pd.read_csv("Data/small_connections_matrix.csv", header=None)
conn_mat = conn_mat_df.values

# Bump each valid index by 1
valid_indices = [i + 1 for i in valid_indices]

# Filter to only incude valid indices
conn_mat = conn_mat[valid_indices, :][:, valid_indices]

total_connection_wd = 0.0
total_connection_passengers = 0

# Load walking distances between gates.
# This CSV is expected to have a "Gate_Name" column and other columns with gate names.
walking_distances_gate_to_gate = pd.read_csv("Data/Walking Distances Gate-to-Gate.csv")

# Loop over flight pairs
num_rows, num_cols = conn_mat.shape
for i in range(1, num_rows):
    for j in range(1, num_cols):
        num_connect = conn_mat[i, j]
        if num_connect > 0:
            # Retrieve flights corresponding to the current indices (skip if index out-of-range)
            try:
                flight_i = df.iloc[i]
                flight_j = df.iloc[j]
            except IndexError:
                continue
            
            # Determine gate assignment based on flight type
            gate_i = flight_i["OptDepGate"] if flight_i["IsDeparting"] == "Y" else flight_i["OptArrGate"]
            gate_j = flight_j["OptDepGate"] if flight_j["IsDeparting"] == "Y" else flight_j["OptArrGate"]

            gate_i_int_match = walking_distances[walking_distances["Gate_Name"] == gate_i]
            if gate_i_int_match.empty:
                print(f"WARNING: Gate {gate_i} not found in walking_distances.")
                continue
            gate_i = gate_i_int_match.iloc[0]["Gate_Int"]

            gate_j_int_match = walking_distances[walking_distances["Gate_Name"] == gate_j]
            if gate_j_int_match.empty:
                print(f"WARNING: Gate {gate_j} not found in walking_distances.")
                continue
            gate_j = gate_j_int_match.iloc[0]["Gate_Int"]
            
            gate_to_gate_wd = conn_mat[i, j]
            total_connection_wd += num_connect * gate_to_gate_wd
            total_connection_passengers += num_connect

avg_connection_wd = total_connection_wd / total_connection_passengers if total_connection_passengers else 0

print("Average Departing Passenger Walking Distance:", avg_departing_wd)
print("Average Arriving Passenger Walking Distance:", avg_arriving_wd)
print("Average Connecting Passenger Walking Distance:", avg_connection_wd)
print("Total Departing Passengers:", total_departing_passengers)
print("Total Arriving Passengers:", total_arriving_passengers)
print("Total Connecting Passengers:", total_connection_passengers)
print("^Passengers are fewer because we filtered out some rows")



Metrics based on actual 2016 gate assignments:
Index(['Gate_Int', 'Gate_Name', 'TSA_to_Gate', 'Gate_to_Bag'], dtype='object')
Rows filtered out because gate assignment was not in terminals A, B, or C: 0
Rows remaining in df: 100
Indices of filtered out rows: []
Average Departing Passenger Walking Distance: 119.62385321100918
Average Arriving Passenger Walking Distance: 173.5531914893617
Average Connecting Passenger Walking Distance: 7.2
Total Departing Passengers: 8175
Total Arriving Passengers: 3525
Total Connecting Passengers: 40
^Passengers are fewer because we filtered out some rows
